# Machine learning

<br><br><br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Solutions to the first project

<br><br><br>

## Scikit-Learn

<img src="img/sklearn-logo.svg" width="300">

In [2]:
import sklearn as sk

<br><br><br>